In [1]:
import tensorflow as tf
from keras.applications import NASNetMobile
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
import keras

In [2]:
# NasNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

NASNetMobile = NASNetMobile(weights=None,include_top=False,input_shape=(img_rows,img_cols,3))

for layer in NASNetMobile.layers:
    layer.trainable = True
    
for (i,layer) in enumerate(NASNetMobile.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer True
1 Conv2D True
2 BatchNormalization True
3 Activation True
4 Conv2D True
5 BatchNormalization True
6 Activation True
7 Activation True
8 ZeroPadding2D True
9 ZeroPadding2D True
10 SeparableConv2D True
11 SeparableConv2D True
12 BatchNormalization True
13 BatchNormalization True
14 Activation True
15 Activation True
16 SeparableConv2D True
17 SeparableConv2D True
18 Activation True
19 BatchNormalization True
20 BatchNormalization True
21 ZeroPadding2D True
22 Activation True
23 Add True
24 SeparableConv2D True
25 ZeroPadding2D True
26 Activation True
27 BatchNormalization True
28 SeparableConv2D True
29 SeparableConv2D True
30 Activation True
31 BatchNormalization True
32 BatchNormalization True
33 ZeroPadding2D True
34 SeparableConv2D True
35 Activation True
36 Activation True
37 MaxPooling2D True
38 BatchNormalization True
39 SeparableConv2D True
40 SeparableConv2D True
41 Activation True
42 Add True
43 AveragePooling2D True
44 BatchNormalization True
45 AveragePoolin

In [3]:
def addTopModelNASNetMobile(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model)
    
    top_model = Dense(num_classes,activation='softmax')(top_model)

    return top_model

num_classes = 3  # ['COVID19','NORMAL','Viral Pneumonia']

FC_Head = addTopModelNASNetMobile(NASNetMobile, num_classes)

model = Model(inputs = NASNetMobile.input, outputs = FC_Head)

print(model.summary())

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           stem_bn1[0][0]                   
_______________________________________________________________________________________

separable_conv_1_bn_normal_left (None, 7, 7, 176)    704         separable_conv_1_normal_left2_11[
__________________________________________________________________________________________________
separable_conv_1_bn_normal_righ (None, 7, 7, 176)    704         separable_conv_1_normal_right2_11
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 7, 7, 176)    704         separable_conv_1_normal_left5_11[
__________________________________________________________________________________________________
activation_166 (Activation)     (None, 7, 7, 176)    0           separable_conv_1_bn_normal_left1_
__________________________________________________________________________________________________
activation_168 (Activation)     (None, 7, 7, 176)    0           separable_conv_1_bn_normal_right1
__________________________________________________________________________________________________
activation

In [4]:
train_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\stacking'
validation_data_dir = r'D:\Dev65\python_workspace2\test\collection-re2\fuzzyColor'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=15,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    horizontal_flip=True,
                    fill_mode='nearest'
                                   )
print(type(train_datagen))
validation_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 16

<class 'tensorflow.python.keras.preprocessing.image.ImageDataGenerator'>


In [5]:

train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols),
                        batch_size = batch_size,
                        class_mode = 'categorical'
                        )

# 경로를 통해 labeling 한 데이터 셋을 만드는 것
validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')


Found 3036 images belonging to 3 classes.
Found 3036 images belonging to 3 classes.


In [6]:
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [ ]:


checkpoint = ModelCheckpoint(
                             'nasNet4.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

# earlystop = EarlyStopping(
#                           monitor='val_loss',
#                           min_delta=0,
#                           patience=10,
#                           verbose=1,restore_best_weights=True)

# learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
#                                             patience=5, 
#                                             verbose=1, 
#                                             factor=0.2, 
#                                             min_lr=0.0001)

#callbacks = [earlystop,checkpoint,learning_rate_reduction]
callbacks = [checkpoint]


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 3036 
nb_validation_samples = 3036 

epochs = 100

history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,     
            epochs=epochs,
            callbacks=callbacks,
            #validation_split=0.2
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size
)

print("끝났다!")

model.save('nasNet4_complete.h5')
model.save_weights('nasNet4_Weight.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
189/189 [==============================] - ETA: 0s - loss: 0.5727 - accuracy: 0.8129
Epoch 00001: val_loss improved from inf to 27.32340, saving model to nasNet4.h5
189/189 [==============================] - 353s 2s/step - loss: 0.5727 - accuracy: 0.8129 - val_loss: 27.3234 - val_accuracy: 0.4418
Epoch 2/100
189/189 [==============================] - ETA: 0s - loss: 0.3490 - accuracy: 0.8834
Epoch 00002: val_loss improved from 27.32340 to 10.90854, saving model to nasNet4.h5
189/189 [==============================] - 338s 2s/step - loss: 0.3490 - accuracy: 0.8834 - val_loss: 10.9085 - val_accuracy: 0.4421
Epoch 3/100
189/189 [==============================] - ETA: 0s - loss: 0.3088 - accuracy: 0.9007
Epoch 00003: val_loss improved from 10.90854 to 5.76733, saving model to nasNet4.h5
189/189 [==============================] - 344s 2s/step - loss: 0.3088 - accuracy: 0.9007 - val_loss: 5.7673 - val_acc

Epoch 29/100
189/189 [==============================] - ETA: 0s - loss: 0.1028 - accuracy: 0.9675
Epoch 00029: val_loss did not improve from 4.92047
189/189 [==============================] - 326s 2s/step - loss: 0.1028 - accuracy: 0.9675 - val_loss: 21.5707 - val_accuracy: 0.4441
Epoch 30/100
189/189 [==============================] - ETA: 0s - loss: 0.1170 - accuracy: 0.9659
Epoch 00030: val_loss did not improve from 4.92047
189/189 [==============================] - 326s 2s/step - loss: 0.1170 - accuracy: 0.9659 - val_loss: 5.0768 - val_accuracy: 0.5397
Epoch 31/100
189/189 [==============================] - ETA: 0s - loss: 0.1089 - accuracy: 0.9682
Epoch 00031: val_loss did not improve from 4.92047
189/189 [==============================] - 325s 2s/step - loss: 0.1089 - accuracy: 0.9682 - val_loss: 5.9292 - val_accuracy: 0.4749
Epoch 32/100
189/189 [==============================] - ETA: 0s - loss: 0.0968 - accuracy: 0.9695
Epoch 00032: val_loss improved from 4.92047 to 4.13664, sa

Epoch 58/100
189/189 [==============================] - ETA: 0s - loss: 0.0837 - accuracy: 0.9722
Epoch 00058: val_loss improved from 2.96282 to 2.33477, saving model to nasNet4.h5
189/189 [==============================] - 330s 2s/step - loss: 0.0837 - accuracy: 0.9722 - val_loss: 2.3348 - val_accuracy: 0.5992
Epoch 59/100
189/189 [==============================] - ETA: 0s - loss: 0.0624 - accuracy: 0.9791
Epoch 00059: val_loss did not improve from 2.33477
189/189 [==============================] - 325s 2s/step - loss: 0.0624 - accuracy: 0.9791 - val_loss: 24.1036 - val_accuracy: 0.4530
Epoch 60/100
189/189 [==============================] - ETA: 0s - loss: 0.0807 - accuracy: 0.9742
Epoch 00060: val_loss did not improve from 2.33477
189/189 [==============================] - 326s 2s/step - loss: 0.0807 - accuracy: 0.9742 - val_loss: 18.1244 - val_accuracy: 0.4537
Epoch 61/100
189/189 [==============================] - ETA: 0s - loss: 0.0634 - accuracy: 0.9791
Epoch 00061: val_loss did

Epoch 87/100
189/189 [==============================] - ETA: 0s - loss: 0.0514 - accuracy: 0.9831
Epoch 00087: val_loss did not improve from 1.88715
189/189 [==============================] - 325s 2s/step - loss: 0.0514 - accuracy: 0.9831 - val_loss: 13.4770 - val_accuracy: 0.5112
Epoch 88/100
189/189 [==============================] - ETA: 0s - loss: 0.0400 - accuracy: 0.9881
Epoch 00088: val_loss did not improve from 1.88715
189/189 [==============================] - 325s 2s/step - loss: 0.0400 - accuracy: 0.9881 - val_loss: 4.5988 - val_accuracy: 0.6088
Epoch 89/100
189/189 [==============================] - ETA: 0s - loss: 0.0612 - accuracy: 0.9781
Epoch 00089: val_loss did not improve from 1.88715
189/189 [==============================] - 325s 2s/step - loss: 0.0612 - accuracy: 0.9781 - val_loss: 21.6745 - val_accuracy: 0.4544
Epoch 90/100
189/189 [==============================] - ETA: 0s - loss: 0.0557 - accuracy: 0.9858
Epoch 00090: val_loss did not improve from 1.88715
189/18